# DMD EDA

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import defaultdict
from pathlib import Path
from pprint import pprint
from typing import Any

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import FuncFormatter

from model.dmd import (
    DRIVER_SESSION_MAPPING,
    ROOT,
    get_frame_paths,
)
from model.fonts import set_cmu_serif_font
from model.memory_map import MemMapReader

set_cmu_serif_font()
plt.rcParams['font.size'] = 20

## List training and test data

In [ ]:
data = defaultdict(dict)

for driver, sessions in DRIVER_SESSION_MAPPING.items():
    for session in sessions:
        session = ROOT / session
        data[driver][session.name + '_normal'] = len(
            get_frame_paths(session, 'normal', 'rgb', 'jpg')
        )
        data[driver][session.name + '_anomal'] = len(
            get_frame_paths(session, 'anomal', 'rgb', 'jpg')
        )

In [ ]:
pprint(data)

In [ ]:
table_data = defaultdict(dict)

for driver, driver_data in data.items():
    for session, frames in driver_data.items():
        if 's1' in session:
            if 'normal' in session:
                table_data[driver]['test_normal'] = frames
            else:
                table_data[driver]['test_anomal'] = frames
        else:
            if 'normal' in session:
                if 'train_normal' in table_data[driver]:
                    table_data[driver]['train_normal'] += frames
                else:
                    table_data[driver]['train_normal'] = frames
            else:
                pass
                # if 'train_anomal' in table_data[driver]:
                #     table_data[driver]['train_anomal'] += frames
                # else:
                #     table_data[driver]['train_anomal'] = frames

table_data_tab = {}
table_data_tab[''] = [
    'Train samples',
    'Test negative samples',
    'Test positive samples',
]
for driver, driver_data in table_data.items():
    frames = [
        driver_data['train_normal'],
        driver_data['test_normal'],
        driver_data['test_anomal'],
    ]
    table_data_tab[f'Driver {driver}'] = frames

In [ ]:
df = pd.DataFrame(table_data_tab)
df.head()

In [ ]:
# Export to LaTeX
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)
df.to_latex(
    output_dir / 'dmd_stats.tex',
    index=False,
    caption=r'Information about train-test split for DMD drivers from group A. Session \texttt{s1} is used for testing, while the rest are used for training.',
    label='tab:dmd-stats',
    position='htb',
)

## Intel Realsense Depth Video

In [ ]:
input_video_path = (
    ROOT
    / 'gA_1_s1_2019-03-08T09;31;15+01;00'
    / 'gA_1_s1_2019-03-08T09;31;15+01;00_depth_body.avi'
)
assert input_video_path.exists()

cap = cv2.VideoCapture(str(input_video_path))
cap.set(cv2.CAP_PROP_CONVERT_RGB, 0)  # Disable RGB conversion
cap.set(cv2.CAP_PROP_POS_FRAMES, 521)
ret, frame = cap.read()
cap.release()

print(frame.min(), frame.max(), frame.dtype, frame.shape)

# Define the threshold (e.g. 2000 mm for 2 meters)
depth_threshold = 2000

# Clip the depth values so that any value above the threshold is set to the threshold
# img_clipped = np.clip(frame, 0, depth_threshold)
img_clipped = np.where(frame > depth_threshold, 0, frame)

# Map the range [0, depth_threshold] to [0, 255]
img8 = ((img_clipped / depth_threshold) * 255).astype(np.uint8)

plt.figure(figsize=(11, 6))
plt.imshow(img8, cmap='gray')
plt.axis('off')
plt.show()


def tick_formatter(x: Any, pos: Any) -> str:
    """Formatter function that scales tick values from 0-255 to 0-2000"""
    return f'{int(round(x * depth_threshold / 255))}'


plt.figure(figsize=(12, 7))
plt.imshow(img8, cmap=plt.cm.inferno)  # type: ignore

cbar = plt.colorbar(shrink=0.75)
cbar.ax.yaxis.set_major_formatter(FuncFormatter(tick_formatter))
vmin, vmax = cbar.mappable.get_clim()
ticks = np.linspace(vmin, vmax, 8)
cbar.set_ticks(ticks)  # type: ignore
cbar.set_label('Depth (mm)', rotation=270, labelpad=30)

plt.axis('off')
plt.tight_layout()
plt.savefig(
    output_dir / 'dmd-depth-sensor.pdf',
    bbox_inches='tight',
    dpi=300,
)

plt.show()

## Illustration of Image Types

In [ ]:
data_path = ROOT / 'gA_1_s1_2019-03-08T09;31;15+01;00/anomal/sequence_2/memory_maps'
output_dir = Path('outputs/source_types')
output_dir.mkdir(exist_ok=True)

In [ ]:
reader_depth = MemMapReader(data_path / 'depth_64.dat', (64, 64))
reader_rgb = MemMapReader(data_path / 'rgb_64.dat', (64, 64, 3))
reader_mask = MemMapReader(data_path / 'masks_64.dat', (64, 64))
reader_sensor = MemMapReader(data_path / 'source_depth_64.dat', (64, 64))

In [ ]:
original_image_path = data_path.parent / 'crop_rgb' / '000521.jpg'
original_image = cv2.imread(str(original_image_path))
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
original_image = cv2.resize(original_image, (512, 512))
original_image = original_image.astype(np.uint8)

In [ ]:
depth_image = reader_depth[0]
rgb_image = reader_rgb[0]
mask_image = reader_mask[0]
sensor_image = reader_sensor[0]

In [ ]:
for img, gray, name in zip(
    [
        original_image,
        mask_image,
        depth_image,
        sensor_image,
        rgb_image,
    ],
    [False, True, True, True, False],
    ['original', 'mask', 'depth', 'depth_sensor', 'rgb'],
    strict=True,
):
    img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_NEAREST)
    plt.figure(figsize=(7, 7))
    plt.imshow(
        img,
        cmap='gray' if gray else None,
    )
    plt.axis('off')
    plt.imsave(
        output_dir / f'{name}.png',
        img,
        cmap='gray' if gray else None,
    )